# Spaceship Titanic NN Model 🌠

Hello Kaggle; 
My goal is to provide the best single **Neuronal Network Model** for the **Space Ship Titanic** competition...
I follow this typical strategy in majority of my Notebook and works quite well for me so I want it to share it.

## Strategy

* Installing Libraries.
* Importing Requiered Libraries
* Setting Notebook Parameters
* Loading the Datasets from CSV to Pandas Dataset
* Exploring the Datasets
* Creating Additional Features
* Pre-Processing
* Feature Selection
* Advance Feature Creation (Clustering)
* Selecting Features for Model Developmetn
* Developing a DNN Model
* Training a Model Utilizing a CV Loop
* Generating Finala Predictions


## File and Data Field Descriptions

train.csv - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
* **PassengerId** - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
* **HomePlanet** - The planet the passenger departed from, typically their planet of permanent residence.
* **CryoSleep** - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
* **Cabin** - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
* **Destination** - The planet the passenger will be debarking to.
* **Age** - The age of the passenger.
* **VIP** - Whether the passenger has paid for special VIP service during the voyage.
* **RoomService**, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
* **Name** - The first and last names of the passenger.
* **Transported** - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

test.csv - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of Transported for the passengers in this set.

sample_submission.csv - A submission file in the correct format.
* **PassengerId** - Id for each passenger in the test set.
* **Transported** - The target. For each passenger, predict either True or False.


## Baseline Model
Hello, here is the location of a basaline model for the competition if you are interested in the modeling component...

https://www.kaggle.com/code/cv13j0/spaceship-my-starter-model


**Last Updates**

Worked on the notebook structure and format.

# 1.0 Installing Required Libraries

In [ ]:
%%capture
!git clone https://github.com/analokmaus/kuma_utils.git

In [ ]:
%%time
import os
import sys
sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer

In [ ]:
%%capture
# Install Gokinjo...
!pip install gokinjo

---

# 2.0 Importing Libraries for the Model

In [ ]:
%%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture
from sklearn.preprocessing import LabelEncoder
from gokinjo import knn_kfold_extract
from gokinjo import knn_extract

In [ ]:
%%time
from sklearn.preprocessing import LabelEncoder 

---

# 3.0 Seeting Notebook Parameters...

In [ ]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
# Notebook Configuration...

# Amount of data we want to load into the Model...
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize...
NROWS = 100
NCOLS = 8
# Main data location path...
BASE_PATH = '...'

In [ ]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)

---

# 4.0 Loading Information from CSV...

In [ ]:
%%time
trn_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
tst_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

sub = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

---

# 5.0 Exploring the Information Available

In [ ]:
%%time
# Explore the shape of the DataFrame...
trn_data.shape

In [ ]:
%%time
# Display the first few rows of the DataFrame...
trn_data.head()

In [ ]:
%%time
# Display the information from the dataset...
trn_data.info()

In [ ]:
%%time
# Checking for empty or NaN values in the dataset by variable...
trn_data.isnull().sum()

---

# 6.0 Feature Engineering...

## 6.1 Filling NaNs by Using EDA Insights (Age)

In [ ]:
%%time
# Filling NaNs Based on Feature Engineering...
def fill_nans_by_age(df, age_limit = 13):
    df['RoomService'] = np.where(df['Age'] <= age_limit, 0, df['RoomService'])
    df['FoodCourt'] = np.where(df['Age'] <= age_limit, 0, df['FoodCourt'])
    df['ShoppingMall'] = np.where(df['Age'] <= age_limit, 0, df['ShoppingMall'])
    df['Spa'] = np.where(df['Age'] <= age_limit, 0, df['Spa'])
    df['VRDeck'] = np.where(df['Age'] <= age_limit, 0, df['VRDeck'])
    
    return df

In [ ]:
%%time
trn_data =  fill_nans_by_age(trn_data)
tst_data =  fill_nans_by_age(tst_data)

---

## 6.2 Filling NaNs by Using EDA Insights (CryoSleep)

In [ ]:
%%time
# Filling NaNs Based on Feature Engineering...
def fill_nans_by_cryo(df, age_limit = 13):
    df['RoomService'] = np.where(df['CryoSleep'] == True, 0, df['RoomService'])
    df['FoodCourt'] = np.where(df['CryoSleep'] == True, 0, df['FoodCourt'])
    df['ShoppingMall'] = np.where(df['CryoSleep'] == True, 0, df['ShoppingMall'])
    df['Spa'] = np.where(df['CryoSleep'] == True, 0, df['Spa'])
    df['VRDeck'] = np.where(df['CryoSleep'] == True, 0, df['VRDeck'])
    
    return df

In [ ]:
%%time
trn_data =  fill_nans_by_cryo(trn_data)
tst_data =  fill_nans_by_cryo(tst_data)

---

## 6.3 Creating Age Groups Using EDA Insights (Age)

In [ ]:
%%time
def age_groups(df, age_limit = 13):
    df['AgeGroup'] = np.where(df['Age'] < age_limit, 0, 1)
    return df

In [ ]:
%%time
trn_data =  age_groups(trn_data)
tst_data =  age_groups(tst_data)

---

## 6.7 Extracting Deck, Cabin Number and Side

In [ ]:
%%time
def cabin_separation(df):
    '''
    Split the Cabin name into Deck, Number and Side
    
    '''
    
    df['CabinDeck'] = df['Cabin'].str.split('/', expand=True)[0]
    df['CabinNum']  = df['Cabin'].str.split('/', expand=True)[1]
    df['CabinSide'] = df['Cabin'].str.split('/', expand=True)[2]
    
    df.drop(columns = ['Cabin'], inplace = True)
    return df

In [ ]:
%%time
trn_data = cabin_separation(trn_data)
tst_data = cabin_separation(tst_data)

---

## 6.8 Extracting Family Name and Name

In [ ]:
%%time
def name_ext(df):
    '''
    Split the Name of the passenger into First and Family...
    
    '''
    
    df['FirstName'] = df['Name'].str.split(' ', expand=True)[0]
    df['FamilyName'] = df['Name'].str.split(' ', expand=True)[1]
    df.drop(columns = ['Name'], inplace = True)
    return df

In [ ]:
%%time
trn_data = name_ext(trn_data)
tst_data = name_ext(tst_data)

---

## 6.9 Creating Age Groups, Based on EDA

In [ ]:
%%time
def age_groups(df, age_limit = 13):
    df['AgeGroup'] = np.where(df['Age'] < age_limit, 0, 1)
    return df

In [ ]:
%%time
trn_data =  age_groups(trn_data)
tst_data =  age_groups(tst_data)

---

## 6.10 Extracting Group

In [ ]:
def extract_group(df):
    '''
    '''
    df['TravelGroup'] =  df['PassengerId'].str.split('_', expand = True)[0]
    df['TravelGroupPos'] =  df['PassengerId'].str.split('_', expand = True)[1]
    return df

In [ ]:
%%time
trn_data = extract_group(trn_data)
tst_data = extract_group(tst_data)

## 6.11 Imputing Using LightGBM

In [ ]:
trn_data.columns

In [ ]:
cols = [
        #'HomePlanet',
        #'CryoSleep',
        #'Destination',
        'Age',
        #'VIP',
        'RoomService',
        'FoodCourt',
        'ShoppingMall',
        'Spa',
        'VRDeck',
        #'AgeGroup',
        #'CabinDeck',
        #'CabinNum',
        #'CabinSide',
        #'FirstName',
        #'FamilyName',
        #'TravelGroup'
       ]

object_cols = [
               #'HomePlanet',
               #'CryoSleep',
               #'Destination',
               #'VIP',
               #'CabinDeck',
               #'CabinNum',
               #'CabinSide',
               #'FirstName',
               #'FamilyName',
               #'TravelGroup'
              ]

In [ ]:
trn_data[cols].isnull().sum()

In [ ]:
trn_data['VIP']

In [ ]:
%%time
object_cols = ['HomePlanet','CryoSleep','Cabin','Destination','VIP','Name']

# Create a fill missing values function using the Simple Imputer
def ml_imputer(df, cols, object_cols):
    '''
    
    '''
    tmp = df.copy(deep = True)
    lgbm_imputer = LGBMImputer(cat_features = object_cols, n_iter = 50)
    tmp[cols] = lgbm_imputer.fit_transform(tmp[cols])
    print('...........', '\n')
    return tmp

In [ ]:
%%time
trn_data = ml_imputer(trn_data, cols, object_cols)
tst_data = ml_imputer(tst_data, cols, object_cols)

In [ ]:
trn_data.isnull().sum()

---

## 6.4 Filling NaNs by Mode and Mean (Using Groups)

In [ ]:
trn_data[trn_data['TravelGroup'] == '2926']

In [ ]:

people_per_group = trn_data.groupby('TravelGroup')['CabinDeck'].nunique().reset_index()
people_per_group.sort_values('CabinDeck', ascending=False)

In [ ]:
people_per_group.max()

In [ ]:
%%time
def fill_missing(df, group = 'TravelGroup',):
    '''
    Fill NaNs values or with mean or most commond value...
    
    '''
    
    tmp = df.copy()
    tmp = tpm[tmp[group].ising()]
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    numeric_tmp = df.select_dtypes(include = numerics)
    categ_tmp = df.select_dtypes(exclude = numerics)
    

    for col in numeric_tmp.columns:
        print(col)
        #df[col] = df[col].fillna(value = df[col].mean())
        df[col] = df[col].fillna(df.groupby(group)[col].transform('mean'))
        
    for col in categ_tmp.columns:
        print(col)
        #df[col] = df[col].fillna(value = df[col].mode()[0])
        #df[col] = df[col].fillna(df.groupby(group)[col].transform(lambda S: S.mode()[0]))
        df[col] = (df.groupby(group)[col].transform(lambda x: x.value_counts().idxmax()))
    print('...')
    
    return df

In [ ]:
%%time
#trn_data =  fill_missing(trn_data)
#tst_data =  fill_missing(tst_data)

## 6.4 Filling NaNs by Mode and Mean

In [ ]:
%%time
def fill_missing(df):
    '''
    Fill NaNs values or with mean or most commond value...
    
    '''
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    numeric_tmp = df.select_dtypes(include = numerics)
    categ_tmp = df.select_dtypes(exclude = numerics)

    for col in numeric_tmp.columns:
        print(col)
        df[col] = df[col].fillna(value = df[col].mean())
        
    for col in categ_tmp.columns:
        print(col)
        df[col] = df[col].fillna(value = df[col].mode()[0])
        
    print('...')
    
    return df

In [ ]:
%%time
trn_data =  fill_missing(trn_data)
tst_data =  fill_missing(tst_data)

In [ ]:
trn_data.isnull().sum()

## 6.5 Calculating Total Expended in the Ship

In [ ]:
%%time
def total_billed(df):
    '''
    Calculates total amount billed in the trip to the passenger... 
    Args:
    Returns:
    
    '''
    
    df['TotalBilled'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']
    return df

In [ ]:
%%time
trn_data = total_billed(trn_data)
tst_data = total_billed(tst_data)

## 6.6 Filling NaNs by Using EDA Insights (TotalBilled)

In [ ]:
%%time
def fill_nans_by_totalspend(df):
    df['CryoSleep'] = np.where(df['TotalBilled'] > 0, False, df['CryoSleep'])
    return df

In [ ]:
%%time
trn_data =  fill_nans_by_totalspend(trn_data)
tst_data =  fill_nans_by_totalspend(tst_data)

---

## 6.11 Calculating Aggregated Features, Based on Cabin Deck

In [ ]:
%%time
Weltiest_Deck = trn_data.groupby('CabinDeck').aggregate({'TotalBilled': 'sum', 'Transported': 'sum', 'CryoSleep': 'sum', 'PassengerId': 'size'}).reset_index()
Weltiest_Deck['AvgSpended'] = Weltiest_Deck['TotalBilled'] / Weltiest_Deck['PassengerId']
Weltiest_Deck['TransportedPercentage'] = Weltiest_Deck['Transported'] / Weltiest_Deck['PassengerId']
Weltiest_Deck['CryoSleepPercentage'] = Weltiest_Deck['CryoSleep'] / Weltiest_Deck['PassengerId']
Weltiest_Deck = Weltiest_Deck.sort_values('AvgSpended', ascending = False)
Weltiest_Deck.head(10)

In [ ]:
%%time
trn_data = trn_data.merge(Weltiest_Deck[['CabinDeck', 'TransportedPercentage', 'AvgSpended']], how = 'left', on = ['CabinDeck'])
tst_data = tst_data.merge(Weltiest_Deck[['CabinDeck', 'TransportedPercentage', 'AvgSpended']], how = 'left', on = ['CabinDeck'])

---

## 6.12 Calulating the Number of Relatives, Using Family Name

In [ ]:
%%time
trn_relatives = trn_data.groupby('FamilyName')['PassengerId'].count().reset_index()
tst_relatives = tst_data.groupby('FamilyName')['PassengerId'].count().reset_index()

In [ ]:
%%time
trn_relatives = trn_relatives.rename(columns = {'PassengerId': 'NumRelatives'})
tst_relatives = tst_relatives.rename(columns = {'PassengerId': 'NumRelatives'})

In [ ]:
%%time
trn_data = trn_data.merge(trn_relatives[['FamilyName', 'NumRelatives']], how = 'left', on = ['FamilyName'])
tst_data = tst_data.merge(tst_relatives[['FamilyName', 'NumRelatives']], how = 'left', on = ['FamilyName'])

---

## 6.13 Calulating the Number of People Traveling Together, Using Traveling Group**

In [ ]:
%%time
trn_relatives = trn_data.groupby('TravelGroup')['PassengerId'].count().reset_index()
tst_relatives = tst_data.groupby('TravelGroup')['PassengerId'].count().reset_index()

In [ ]:
%%time
trn_relatives = trn_relatives.rename(columns = {'PassengerId': 'GroupSize'})
tst_relatives = tst_relatives.rename(columns = {'PassengerId': 'GroupSize'})

In [ ]:
%%time
trn_data = trn_data.merge(trn_relatives[['TravelGroup', 'GroupSize']], how = 'left', on = ['TravelGroup'])
tst_data = tst_data.merge(tst_relatives[['TravelGroup', 'GroupSize']], how = 'left', on = ['TravelGroup'])

---

# 7.0 Pre-Processing for Training

## 7.1 Separating the Fields by Type

In [ ]:
trn_data.columns

In [ ]:
%%time
# A list of the original variables from the dataset
numerical_features = [
                      'Age', 
                      'RoomService', 
                      'FoodCourt', 
                      'ShoppingMall', 
                      'Spa', 
                      'VRDeck', 
                      'TotalBilled',
                      'TravelGroupPos'
                     ]

categorical_features = [
                        #'Name',
                        'FirstName',
                        'FamilyName',
                        'CabinNum',
                        'TravelGroup',
                        'AgeGroup'
                       ]


categorical_features_onehot = [
                               'HomePlanet',
                               'CryoSleep',
                               #'Cabin',
                               'CabinDeck',
                               'CabinSide',
                               'Destination',
                               'VIP',
                               #'AgeGroup'
                               ]

target_feature = 'Transported'

---

## 7.2 Encoding Categorical Variables

In [ ]:
%%time

def encode_categorical(train_df, test_df, categ_feat = categorical_features):
    '''
    
    '''
    encoder_dict = {}
    
    concat_data = pd.concat([trn_data[categ_feat], tst_data[categ_feat]])
    
    for col in concat_data.columns:
        print('Encoding: ', col, '...')
        encoder = LabelEncoder()
        encoder.fit(concat_data[col])
        encoder_dict[col] = encoder

        train_df[col + '_Enc'] = encoder.transform(train_df[col])
        test_df[col + '_Enc'] = encoder.transform(test_df[col])
    
    train_df = train_df.drop(columns = categ_feat, axis = 1)
    test_df = test_df.drop(columns = categ_feat, axis = 1)

    return train_df, test_df

In [ ]:
%%time
trn_data, tst_data = encode_categorical(trn_data, tst_data, categorical_features)

---

## 7.3 One Hot Encoding Categorical Variables

In [ ]:
%%time
def one_hot(df, one_hot_categ):
    for col in one_hot_categ:
        tmp = pd.get_dummies(df[col], prefix = col)
        df = pd.concat([df, tmp], axis = 1)
    df = df.drop(columns = one_hot_categ)
    return df

In [ ]:
%%time
trn_data = one_hot(trn_data, categorical_features_onehot) 
tst_data = one_hot(tst_data, categorical_features_onehot) 

---

# 8.0 Feature Selection for Baseline Model

In [ ]:
%%time
remove = ['PassengerId', 
          'Route', 
          'FirstName_Enc', 
          #'CabinNum_Enc', 
          'Transported',
          'Cabin',
          'TransportedPercentage',
          #'IsKid', 
          #'IsAdult', 
          #'IsOlder'
          #'RoomService',
          #'FoodCourt',
          #'ShoppingMall',
          #'Spa',
          #'VRDeck',
         ]
features = [feat for feat in trn_data.columns if feat not in remove]

In [ ]:
%%time
features

---

# 9.0 Advance Feature Engineering, KNN

In [ ]:
%%time
# Convert X and y to Numpy arrays as library requirements
X_array = trn_data[features].to_numpy()
y_array = trn_data['Transported'].to_numpy()
X_test_array = tst_data[features].to_numpy()

In [ ]:
K = 2

In [ ]:
%%time
# It Takes almost  35min 21s for K = 2 and 50_000 rows...
# It Takes almost  17min 36s for K = 1 and 50_000 rows...
KNN_trn_features = knn_kfold_extract(X_array, y_array, k = K, normalize = 'standard')

In [ ]:
%%time
KNN_trn_features

In [ ]:
%%time
knn_cols = ['KNN_K1_01',
            'KNN_K1_02',
            'KNN_K2_01',
            'KNN_K2_02']

KNN_feat = pd.DataFrame(KNN_trn_features, columns = knn_cols)
KNN_feat = pd.DataFrame(KNN_trn_features, columns = knn_cols).set_index(trn_data.index)

In [ ]:
%%time
trn_data = pd.concat([trn_data, KNN_feat], axis = 1)
trn_data.head()

In [ ]:
%%time
KNN_tst_features = knn_extract(X_array, y_array, X_test_array, k = K, normalize = 'standard')
KNN_feat = pd.DataFrame(KNN_tst_features, columns = knn_cols).set_index(tst_data.index)

tst_data = pd.concat([tst_data, KNN_feat], axis = 1)
tst_data.head()

---

# 10.0 Selection of Features for Training Stage

In [ ]:
%%time
remove = ['PassengerId', 
          'Route', 
          'FirstName_Enc', 
          'CabinNum_Enc', 
          'Transported',
          'Cabin',
          'TransportedPercentage',
          #'IsKid', 
          #'IsAdult', 
          #'IsOlder'
          #'RoomService',
          #'FoodCourt',
          #'ShoppingMall',
          #'Spa',
          #'VRDeck',
          'KNN_K2_02',
          'KNN_K2_01',
         ]
features = [feat for feat in trn_data.columns if feat not in remove]

In [ ]:
%%time
features

In [ ]:
features = ['Age',
            'RoomService',
            'FoodCourt',
            'ShoppingMall',
            'Spa',
            'VRDeck',
            'TravelGroupPos',
            'TotalBilled',
            #'AvgSpended',
            #'NumRelatives',
            #'GroupSize',
            'FamilyName_Enc',
            'TravelGroup_Enc',
            #'AgeGroup_Enc',
            'HomePlanet_Earth',
            'HomePlanet_Europa',
            'HomePlanet_Mars',
            'CryoSleep_False',
            'CryoSleep_True',
            'CabinDeck_A',
            'CabinDeck_B',
            'CabinDeck_C',
            'CabinDeck_D',
            'CabinDeck_E',
            'CabinDeck_F',
            'CabinDeck_G',
            'CabinDeck_T',
            'CabinSide_P',
            'CabinSide_S',
            'Destination_55 Cancri e',
            'Destination_PSO J318.5-22',
            'Destination_TRAPPIST-1e',
            'VIP_False',
            'VIP_True',
            #'AgeGroup_0',
            #'AgeGroup_1',
            #'AgeGroup_0',
            #'AgeGroup_1',
            'KNN_K1_01',
            #'KNN_K1_02'
]

---

# 11.0 Building a Neuronal Network Model...

## 11.1 Importing all the required libraries...

In [ ]:
%%time
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, BatchNormalization, Dropout, Concatenate

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import random

from sklearn.model_selection import KFold, StratifiedKFold 
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
import datetime
import math

---

## 11.2 Defining the Model Architecture...

In [ ]:
%%time
def nn_model_one():
    '''
    '''
    
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x = Dense(1024, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(inputs)
    
    x = BatchNormalization()(x)
    
    x = Dense(256, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)

    x = Dense(128, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)

    x = Dense(1 , 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(30e-6),
              activation = 'sigmoid')(x)
    
    model = Model(inputs, x)
    
    return model

In [ ]:
%%time
def nn_model_two():
    '''
    '''

    dropout_value = 0.025
    
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x = Dense(1024, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(inputs)
    
    x = BatchNormalization()(x)
    x = Dropout(dropout_value)(x)
    
    x = Dense(256, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)
    x = Dropout(dropout_value)(x)

    x = Dense(128, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)
    x = Dropout(dropout_value)(x)
    
    x = Dense(8, 
          #use_bias  = True, 
          kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
          activation = activation_func)(x)
    
    x = BatchNormalization()(x)
    x = Dropout(dropout_value)(x)
    
    x = Dense(1 , 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(30e-6),
              activation = 'sigmoid')(x)
    
    model = Model(inputs, x)
    
    return model

In [ ]:
%%time
def nn_model_three():
    
    '''
    Function to define the Neuronal Network architecture...
    '''
    
    L2 = 65e-6
    dropout_value = 0.025
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x0 = Dense(1024, kernel_regularizer = tf.keras.regularizers.l2(L2), activation = activation_func)(inputs)
    x0 = BatchNormalization()(x0)
    x0 = Dropout(dropout_value)(x0)
    
    x1 = Dense(1024,  kernel_regularizer = tf.keras.regularizers.l2(L2), activation = activation_func)(x0)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(dropout_value)(x1)
    
    x1 = Dense(64,  kernel_regularizer = tf.keras.regularizers.l2(L2), activation = activation_func)(x1)
    x1 = Concatenate()([x1, x0])
    x1 = BatchNormalization()(x1)
    x1 = Dropout(dropout_value)(x1)
    
    x1 = Dense(16, kernel_regularizer = tf.keras.regularizers.l2(L2), activation = activation_func)(x1)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(dropout_value)(x1)
    
    x1 = Dense(1,  
               #kernel_regularizer = tf.keras.regularizers.l2(4e-4), 
               activation = 'sigmoid')(x1)
    
    model = Model(inputs, x1)
    
    return model

---

## 11.3 Visualizing the Architecture Created...

In [ ]:
%%time
architecture = nn_model_one()
architecture.summary()

---

## 11.4 Defining Model Parameters for Training...

In [ ]:
%%time
# Defining model parameters...
BATCH_SIZE         = 128
EPOCHS             = 250 
EPOCHS_COSINEDECAY = 250
DIAGRAMS           = True
USE_PLATEAU        = True
INFERENCE          = False
VERBOSE            = 0 
TARGET             = 'Transported'

---

## 11.5 Defining Training Functions for the NN Model

In [ ]:
 %%time
# Defining model training function...
def fit_model(X_train, y_train, X_val, y_val, run = 0):
    '''
    '''
    lr_start = 0.2
    start_time = datetime.datetime.now()
    
    #scaler = StandardScaler()
    #scaler = RobustScaler()
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)

    epochs = EPOCHS    
    lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 8, verbose = VERBOSE)
    es = EarlyStopping(monitor = 'val_loss',patience = 16, verbose = 1, mode = 'min', restore_best_weights = True)
    tm = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [lr, es, tm]
    
    # Cosine Learning Rate Decay
    if USE_PLATEAU == False:
        epochs = EPOCHS_COSINEDECAY
        lr_end = 0.0002

        def cosine_decay(epoch):
            if epochs > 1:
                w = (1 + math.cos(epoch / (epochs - 1) * math.pi)) / 2
            else:
                w = 1
            return w * lr_start + (1 - w) * lr_end
        
        lr = LearningRateScheduler(cosine_decay, verbose = 0)
        callbacks = [lr, tm]
        
    model = nn_model_one()
    
    optimizer_func = tf.keras.optimizers.Adam(learning_rate = lr_start)
    loss_func = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer = optimizer_func, loss = loss_func)
    
    X_val = scaler.transform(X_val)
    validation_data = (X_val, y_val)
    
    history = model.fit(X_train, 
                        y_train, 
                        validation_data = validation_data, 
                        epochs          = epochs,
                        verbose         = VERBOSE,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True,
                        callbacks       = callbacks
                       )
    
    history_list.append(history.history)
    print(f'Training loss:{history_list[-1]["loss"][-1]:.3f}')
    callbacks, es, lr, tm, history = None, None, None, None, None
    
    
    y_val_pred = model.predict(X_val, batch_size = BATCH_SIZE, verbose = VERBOSE)
    y_val_pred = [1 if x > 0.5 else 0 for x in y_val_pred]
    
    score = accuracy_score(y_val, y_val_pred)
    print(f'Fold {run}.{fold} | {str(datetime.datetime.now() - start_time)[-12:-7]}'
          f'| ACC: {score:.5f}')
    
    score_list.append(score)
    
    tst_data_scaled = scaler.transform(tst_data[features])
    tst_pred = model.predict(tst_data_scaled)
    predictions.append(tst_pred)
    
    return model

---

# Training a XGBoost Classifier ...

# 12.0 Training the NN Model, Using a CV Loop

In [ ]:
%%time
# Create empty lists to store NN information...
history_list = []
score_list   = []
predictions  = []

# Define kfolds for training purposes...
#kf = KFold(n_splits = 5)
kf = StratifiedKFold(n_splits = 5, random_state = 15, shuffle = True)

for fold, (trn_idx, val_idx) in enumerate(kf.split(trn_data, trn_data[TARGET])):
    X_train, X_val = trn_data.iloc[trn_idx][features], trn_data.iloc[val_idx][features]
    y_train, y_val = trn_data.iloc[trn_idx][TARGET], trn_data.iloc[val_idx][TARGET]
    
    fit_model(X_train, y_train, X_val, y_val)
    
print(f'OOF AUC: {np.mean(score_list):.5f}')

In [ ]:
# OOF AUC: 0.80375
# OOF AUC: 0.80433
# OOF AUC: 0.80536
# OOF AUC: 0.80778
# OOF AUC: 0.80812 ***
# OOF AUC: 0.79420
# OOF AUC: 0.79133

# OOF AUC: 0.79731
# OOF AUC: 0.79708

# OOF AUC: 0.79973 ***

# OOF AUC: 0.79777
# OOF AUC: 0.80387 ***
# OOF AUC: 0.80191 
# OOF AUC: 0.80364 ...

# OOF AUC: 0.80755 >>> 0.81061
# OOF AUC: 0.80421 >>> 0.80664
# OOF AUC: 0.80778 >>> 0.80664
# OOF AUC: 0.80904 >>>

---

# 13.0 Generating Predictions

In [ ]:
%%time
# Populated the prediction on the submission dataset and creates an output file
sub['Transported'] = np.array(predictions).mean(axis = 0)
sub['Transported'] = np.where(sub['Transported'] > 0.5, True, False)
sub.to_csv('my_submission_051922.csv', index = False)


In [ ]:
%%time
sub.head()

---